In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour >= current_hour)

    return query.read()

In [4]:
df = fetch_hourly_rides(12)

2025-04-24 19:33:04,483 INFO: Initializing external client
2025-04-24 19:33:04,484 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-04-24 19:33:05,312 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.85s) 


In [5]:
df

,pickup_hour,pickup_location_id,rides
0,2025-04-24 22:00:00+00:00,201,0
1,2025-04-24 17:00:00+00:00,208,0
2,2025-04-24 07:00:00+00:00,72,5
3,2025-04-24 11:00:00+00:00,126,1
4,2025-04-24 21:00:00+00:00,194,0
...,...,...,...
4075,2025-04-24 19:00:00+00:00,43,85
4076,2025-04-24 14:00:00+00:00,200,0
4077,2025-04-24 17:00:00+00:00,156,0
4078,2025-04-24 15:00:00+00:00,156,0


In [6]:
df_pred = fetch_predictions(240)

2025-04-24 19:33:13,236 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-04-24 19:33:13,242 INFO: Initializing external client
2025-04-24 19:33:13,242 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-04-24 19:33:13,968 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214677
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.44s) 


In [7]:
df_pred

,pickup_location_id,predicted_demand,pickup_hour


In [8]:
merged_df = pd.merge(df, df_pred, on=['pickup_location_id', 'pickup_hour'])

In [9]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand


In [10]:
pd.to_datetime(df['pickup_hour']).dt.floor('h')

0      2025-04-24 22:00:00+00:00
1      2025-04-24 17:00:00+00:00
2      2025-04-24 07:00:00+00:00
3      2025-04-24 11:00:00+00:00
4      2025-04-24 21:00:00+00:00
                  ...           
4075   2025-04-24 19:00:00+00:00
4076   2025-04-24 14:00:00+00:00
4077   2025-04-24 17:00:00+00:00
4078   2025-04-24 15:00:00+00:00
4079   2025-04-24 17:00:00+00:00
Name: pickup_hour, Length: 4080, dtype: datetime64[us, Etc/UTC]

In [11]:
merged_df['difference'] = merged_df['predicted_demand'] - merged_df['rides']

In [12]:
merged_df.sort_values(["pickup_location_id", "pickup_hour"])

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [13]:
merged_df

,pickup_hour,pickup_location_id,rides,predicted_demand,difference


In [14]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

df['pickup_hour'] = pd.to_datetime(df['pickup_hour']).dt.floor('h')
df_pred['pickup_hour'] = pd.to_datetime(df_pred['pickup_hour']).dt.floor('h')


# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['pickup_location_id', 'pickup_hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['rides'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('pickup_hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='pickup_hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'pickup_hour': 'Pickup Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()

In [15]:
mae_by_hour["MAE"].mean()

nan